In [62]:
# Used libraries 
import pandas as pd
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
#import spacy
import re




In [63]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Load Spacy model
nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mervetaskaya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


NameError: name 'spacy' is not defined

In [71]:
def read_data():
    df_artists = pd.read_csv('data/spotify_artists.csv', sep=',') 
    df_tracks = pd.read_csv('data/spotify_tracks.csv', sep=',')   
    return df_artists, df_tracks 
     
df_artists, df_tracks = read_data()

In [66]:
pd.set_option('display.max_columns', None)
df_artists.head(5)  


,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo trad...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
1,1,22,313,[],1dLnVku4VQUOLswwDFvRc9,The Grenadines,4wqwj0gA8qPZKLl5WVqXml,track_30,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,['uk alternative pop'],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,['french baroque'],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist


In [67]:
#check in df_artists if we have duplicates via id
df_artists[df_artists.duplicated(['id'])]

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type


In [72]:
#check if there are missing genres
df_artists['genres'].value_counts()

#remove the rows with missing genres
df_artists = df_artists[df_artists['genres'] != '[]']
df_artists

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo trad...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,['uk alternative pop'],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,['french baroque'],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist
9,9,25,992,['classic finnish pop'],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist
...,...,...,...,...,...,...,...,...,...
56124,56124,23,1212,['chilean rock'],48YkQQTWzmT248VY8DF7Fv,Profetas y Freneticos,29WK73lMQpLELfHd6PfbjY,track_20,artist
56125,56125,42,57973,"['aggro chileno', 'andean', 'chilean rock', 'l...",1OgaghfYd1qr8DFoNrfDNk,Sinergia,0rXhmIYRPIoVYstJdZt4uq,track_21,artist
56126,56126,38,20870,"['australian alternative rock', 'australian ro...",7o9kdTx6RmO12iAVVsNehd,The Superjesus,1B7tV3WzEnDUS7wXSKDXk3,track_1,artist
56127,56127,44,33923,"['australian alternative rock', 'australian ro...",6n3YUZcayLRuAunJUUelvz,Regurgitator,4e5wI6VC4eVDTtpyZ409Pw,track_8,artist


In [73]:
#convert the string to list

import ast

df_artists['genres'] = df_artists['genres'].apply(ast.literal_eval)
df_artists.head(5)    

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"[sertanejo, sertanejo pop, sertanejo tradicion...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
2,2,26,1596,[danish pop rock],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist
3,3,31,149,[uk alternative pop],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist
4,4,21,11,[french baroque],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist
9,9,25,992,[classic finnish pop],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist


In [75]:
df_artists['cleaned_genres'] = df_artists['genres'].apply(lambda x: [re.sub(r'[^a-zA-Z\s]', '', genre) for genre in x])
df_artists.head(5)

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type,cleaned_genres
0,0,44,23230,"[sertanejo, sertanejo pop, sertanejo tradicion...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist,"[sertanejo, sertanejo pop, sertanejo tradicion..."
2,2,26,1596,[danish pop rock],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist,[danish pop rock]
3,3,31,149,[uk alternative pop],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist,[uk alternative pop]
4,4,21,11,[french baroque],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist,[french baroque]
9,9,25,992,[classic finnish pop],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist,[classic finnish pop]


### Preprocessing | Text standardization or normalization

In [56]:
def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Entfernung von Satzzeichen (Klammern und Anführungszeichen)
    text = re.sub(r'\[|\]|\'|', '', text)    

    return text

# Anwendung der Textvorverarbeitung auf Ihre Daten
df_artists['genres_new'] = df_artists['genres'].apply(preprocess_text)
df_artists.head(5)

AttributeError: 'list' object has no attribute 'lower'

### Preprocessing | Spelling Correction

### Tokenization

In [77]:
df_artists['tokenized_genre'] = df_artists['cleaned_genres'].apply(lambda x: [nltk.word_tokenize(genre) for genre in x])
df_artists.head(5)

,Unnamed: 0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type,cleaned_genres,tokenized_genre
0,0,44,23230,"[sertanejo, sertanejo pop, sertanejo tradicion...",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist,"[sertanejo, sertanejo pop, sertanejo tradicion...","[[sertanejo], [sertanejo, pop], [sertanejo, tr..."
2,2,26,1596,[danish pop rock],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist,[danish pop rock],"[[danish, pop, rock]]"
3,3,31,149,[uk alternative pop],2VElyouiCfoYPDJluzwJwK,FADES,3MFSUBAidPzRBbIS7BDj1S,track_34,artist,[uk alternative pop],"[[uk, alternative, pop]]"
4,4,21,11,[french baroque],4agVy03qW8juSysCTUOuDI,Jean-Pierre Guignon,2r3q57FhxdsCyYr0kuDq4b,track_26,artist,[french baroque],"[[french, baroque]]"
9,9,25,992,[classic finnish pop],5ijoPCUhV0dW8EJ7aPGvXK,Pepe Willberg & The Paradise,6WVlVPoesX2zVvJzZJUj6a,track_8,artist,[classic finnish pop],"[[classic, finnish, pop]]"
